In [1]:

import pandas as pd
import numpy as np
import keras
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

In [2]:
data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())

                                                Text Emotions
0  i can go from feeling so hopeless to so damned...  sadness
1   im grabbing a minute to post i feel greedy wrong    anger
2  i am ever feeling nostalgic about the fireplac...     love
3                               i am feeling grouchy    anger
4  ive been feeling a little burdened lately wasn...  sadness


In [3]:

texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [4]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [5]:
sequences = tokenizer.texts_to_sequences(texts)

In [6]:

padded_sequences

array([[   0,    0,    0, ...,    3,   21, 1254],
       [   0,    0,    0, ...,    2,  494,  437],
       [   0,    0,    0, ...,   29,    5, 3496],
       ...,
       [   0,    0,    0, ...,    3,  101, 1331],
       [   0,    0,    0, ...,  339,    8,   42],
       [   0,    0,    0, ...,   25, 3585,   12]])

In [7]:

# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [8]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)


In [9]:

one_hot_labels

array([[0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.]])

In [10]:

# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, 
                                                one_hot_labels, 
                                                test_size=0.2)

In [11]:

one_hot_labels[0]

array([0., 0., 0., 0., 1., 0.])

In [12]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))


Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 24s 55ms/step - accuracy: 0.3899 - loss: 1.5259 - val_accuracy: 0.7312 - val_loss: 0.8308
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 21s 53ms/step - accuracy: 0.8803 - loss: 0.4022 - val_accuracy: 0.8266 - val_loss: 0.5244
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 22s 56ms/step - accuracy: 0.9881 - loss: 0.0508 - val_accuracy: 0.8178 - val_loss: 0.5677
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 21s 53ms/step - accuracy: 0.9957 - loss: 0.0231 - val_accuracy: 0.8241 - val_loss: 0.6206
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 23s 57ms/step - accuracy: 0.9962 - loss: 0.0158 - val_accuracy: 0.8150 - val_loss: 0.6443
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 24s 59ms/step - accuracy: 0.9970 - loss: 0.0128 - val_accuracy: 0.8169 - val_loss: 0.7005
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 25s 64ms/step - accuracy: 0.9978 - loss: 0.0118 - val_accuracy: 0.8138 - val_loss: 0.7302
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 24s 61ms/step - accuracy: 0.9983 - loss: 0.0095 - 

In [13]:
model.save("trained_model.h5")

In [14]:
def predict_label(input_text, tokenizer, max_length, model, label_encoder):
    # Preprocess the input text
    input_sequence = tokenizer.texts_to_sequences([input_text])
    padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
    prediction = model.predict(padded_input_sequence)
    predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
    return predicted_label[0]

# Example usage
input_text = "i am ever feeling nostalgic about the fireplace i will know that it is still on the property"
predicted_label = predict_label(input_text, tokenizer, max_length, model, label_encoder)
print(predicted_label)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
love


In [16]:
import pickle
# Extract the maximum length from the tokenizer
max_length = max([len(seq) for seq in sequences])

# Save the tokenizer
with open("tokenizer.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save label encoder
with open("label_encoder.pickle", "wb") as handle:
    pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save the maximum length
with open("max_length.pickle", "wb") as handle:
    pickle.dump(max_length, handle, protocol=pickle.HIGHEST_PROTOCOL)
